### Equality Constrained Optimization

Problem Setup:
$$ \min_{x \in \mathbb{R}^n} f(x) \\ \text{s.t. }  g(x)=0$$
where
$f\colon \mathbb{R}^n \rightarrow \mathbb{R} $ and $g\colon \mathbb{R}^n \rightarrow \mathbb{R}^m $ 

We define the Lagrangian as: $\mathcal{L}(x,\mathbf{\lambda}) = f(x) - \mathbf{\lambda}' g(x)$.

The KKT conditions (First Order Necesary Conditions):
$$\nabla\mathcal{L}(x,\mathbf{\lambda}) = 0 $$
$$g(x) = 0$$.

For a general nonlinear problem, we can use Newton-optimization methods to solve the above system of equation.
This leads to essentially solving the following linear system of equation at every iteration of the Newton-type optimization:

$$ $$
$$
 \begin{bmatrix} \nabla_x f(x_k) \\  g(x_k) \end{bmatrix} + \begin{bmatrix} \nabla^2\mathcal{L} & \nabla g \\ \nabla g' & 0 \end{bmatrix} \begin{bmatrix} (x_{k+1}-x_k) \\ -\lambda_{k+1} \end{bmatrix}
 = 0
$$

Different approximations to the matrix $\bf{B} = \nabla^2\mathcal{L}$ results in different Newton-type optimization algorithms for the equality constrained optimization.

For the special case of QP, $f(x) = \frac{1}{2} x' \mathbf{B} x + c'x$, $g(x) = b + \mathbf{A}x$.
then the KKT conditions directly lead to the following solution for the optima:
$$\begin{bmatrix} \mathbf{B} & \mathbf{A}' \\ \mathbf{A}& 0 \end{bmatrix} \begin{bmatrix} x \\ -\lambda \end{bmatrix} + \begin{bmatrix}c \\ b\end{bmatrix}=0$$

In [332]:
# QP: Linear equality constraint and quadratic cost
import numpy as np
def QP(B,c,A,b, method=None):
    '''
    OBJECTIVE; min (0.5*x'Bx+c'x) s.t. Ax+b=0 
    Assuming Second Order Sufficiency Condition holds. i.e., p'Bp >0 for all p in the kernel(A)
    
    We solve the above linear equation for QP solution in a smart way usin Null-space method
    (instead of naive approaches of taking the inversion of KKT matrix). 
    The naive method puts stronger constraints on (A,B) for QP solution to exist.
    '''
    if method == "NullSpace":
        # Solve using null space method
        q, r  = np.linalg.qr(A.T,mode='complete')
        print(r)
        print(q)
        Z = q[:,m:]#vh[m:,:].T # basis for kernel(A)
        Y = q[:,:m]
        Rp = r[:m,:]
        x_special = np.linalg.lstsq(A, -1*np.array(b).reshape(-1,))[0]
        a_ = Z.T@B@Z 
        b_ = Z.T@np.array(c).reshape(-1,)+Z.T@B@np.array(x_special).reshape(-1,)
        x_g = np.linalg.lstsq(a_, -1*np.array(b_).reshape(-1,))[0]
        x = (Z@x_g + x_special).reshape(-1,)
        lambda_ = np.linalg.lstsq(Rp, -Y.T@(B@x+c.reshape(-1,)))[0] #lagrangian
    
    else:
        #naive method: faster but requirement on (A,B) is stronger for solution to exist
        nA_ = np.zeros([n+m,n+m])
        nb_ = np.zeros(n+m)
        nA_[:n,:n] = B
        nA_[n:,:n] = A
        nA_[:n,n:] = A.T
        nb_[:n] = -c
        nb_[n:] = -b
        x_lambda = np.linalg.lstsq(nA_, np.array(nb_).reshape(-1,))[0]
        x = x_lambda[:n]
        lambda_ = x_lambda[n:] #lagrangian
                            
    return x,lambda_
    

In [334]:
# Example QP:
n = 7
m = 2
A = np.random.randn(m,n)
B = np.random.randn(n,n)
c = np.random.randn(n)
b = np.random.randn(m)
x,lambda_ = QP(B,c,A,b)
print("Minima, x= ",x)
print("lambda (lagrangian) = ",lambda_)
print("Minimum value = ",0.5*x.T@B@x +c.T@x)

Minima, x=  [-0.73069497 -0.25064868 -0.5167219   0.0861169   0.29881158  0.54086647
  0.91652219]
lambda (lagrangian) =  [ 0.05113968 -0.19032848]
Minimum value =  0.6423165347503986


<ipython-input-332-b6330cce5964>:37: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x_lambda = np.linalg.lstsq(nA_, np.array(nb_).reshape(-1,))[0]
